## Regression Model for USDA Beef Grading

This model will use pytoch and resnet to make a CNN regression model (convolutional neural network)
hay que mantener 5 pasos
1. cargando imagenes y procesarlos
2. modificar el modelo resnet para la regresion
3. entrenar el model
4. evaluar modelo (root mean square)

In [169]:
import torch
import torchvision
import os
import torch.nn as nn
import numpy as np
from torchvision import datasets, transforms, models
import torch.optim as optim
import pandas as pd
from torchvision.models import ResNet18_Weights
from torch.utils.data import Dataset, DataLoader
from torch.optim.lr_scheduler import StepLR

from PIL import Image


from sklearn.metrics import mean_squared_error

print(torch.cuda.is_available())
print(torch.version.cuda)

True
12.4


In [170]:
image_folder = "Data/Cleaned JPEG Images/Cargill Dodge 07 20 22 Plant/Images"
excel_path = "Data/Cleaned JPEG Images/Cargill Dodge 07 20 22 Plant/data.xlsx"

df = pd.read_excel(excel_path)

df.columns = df.columns.str.strip()
#ID_to_score = dict(zip(df['CarcassId'], df['Average']))
ID_to_score = {f"{carcass_id}.jpg": avg for carcass_id, avg in zip(df['CarcassId'], df['Average'])}

#print(ID_to_score.keys())

class ImageDataset(Dataset):
    def __init__(self, image_dir, transform =None):
        self.image_dir = image_dir
        self.transform = transform
        self.image_filenames = list(ID_to_score.keys())
        
    def __len__(self):
        return len(self.image_filenames)
        
    def __getitem__(self, idx):
        img_name = os.path.join(self.image_dir, self.image_filenames[idx])
        image = Image.open(img_name)
        score = ID_to_score[self.image_filenames[idx]]
        if self.transform:
            image = self.transform(image)
        return image, score
      

# Transforms and model

In [171]:
transform = transforms.Compose([
    transforms.Resize((224,224)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406], [0.229,0.224,0.225]),
])

dataset = ImageDataset(image_folder, transform=transform)
dataloader= DataLoader(dataset, batch_size=32, shuffle=True)

In [172]:
   #load pretrained resnet model
resnet = models.resnet18(weights=ResNet18_Weights.DEFAULT)

num_features = resnet.fc.in_features
resnet.fc = nn.Linear(num_features,1) #output 1 for regression

#LOAD IN THE GPU (MUY IMPORTANTE NO OLVIDES)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
resnet = resnet.to(device)

cuda


# Training Loop

In [173]:
criterion = nn.MSELoss()
optimizer = optim.Adam(resnet.parameters(), lr = 0.001)
scheduler = StepLR(optimizer, step_size=40, gamma=0.1)

num_epochs = 65

for epoch in range(num_epochs):
    running_loss =0.0
    for images, scores in dataloader:
        images = images.to(device)
        scores = scores.to(device).view(-1,1).float()
        
        optimizer.zero_grad()
        
        outputs = resnet(images)
        loss = criterion(outputs, scores)
        
        #backward pass and optimization
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        
    scheduler.step()
    
    print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(dataloader):.4f}')
    
print('Training completed!')

Epoch [1/65], Loss: 182630.7691
Epoch [2/65], Loss: 175358.0590
Epoch [3/65], Loss: 173416.7361
Epoch [4/65], Loss: 166824.3038
Epoch [5/65], Loss: 161668.0208
Epoch [6/65], Loss: 158619.5521
Epoch [7/65], Loss: 152270.8750
Epoch [8/65], Loss: 145265.3802
Epoch [9/65], Loss: 138760.8733
Epoch [10/65], Loss: 132180.5373
Epoch [11/65], Loss: 124426.1970
Epoch [12/65], Loss: 118560.6510
Epoch [13/65], Loss: 111897.7569
Epoch [14/65], Loss: 104566.4106
Epoch [15/65], Loss: 98065.2439
Epoch [16/65], Loss: 90692.0026
Epoch [17/65], Loss: 85449.9618
Epoch [18/65], Loss: 78205.3880
Epoch [19/65], Loss: 71826.1432
Epoch [20/65], Loss: 66184.8650
Epoch [21/65], Loss: 59812.9666
Epoch [22/65], Loss: 54018.3277
Epoch [23/65], Loss: 49244.5543
Epoch [24/65], Loss: 43684.5747
Epoch [25/65], Loss: 38916.6376
Epoch [26/65], Loss: 34297.5601
Epoch [27/65], Loss: 31275.2339
Epoch [28/65], Loss: 26628.8257
Epoch [29/65], Loss: 23772.1467
Epoch [30/65], Loss: 19936.9427
Epoch [31/65], Loss: 17755.9965
Epo

# Evaluation

In [174]:
def evaluate_model(model, dataloader):
    model.eval()
    all_preds = []
    all_scores = []
    with torch.no_grad():
        for images, scores in dataloader:
            images = images.to(device)
            scores = scores.to(device).view(-1,1).float()
            outputs = model(images)
            all_preds.append(outputs.cpu().numpy())
            all_scores.append(scores.cpu().numpy())
            
    all_preds = np.concatenate(all_preds)
    all_scores = np.concatenate(all_scores)
    mse = mean_squared_error(all_scores, all_preds)
    rmse = np.sqrt(mse)
    
    print(f'RMSE: {rmse:.4f}')
    return rmse

evaluate_model(resnet, dataloader)

RMSE: 31.2301


31.230062

# Misc / Junk Code